In [66]:
import pandas as pd
import numpy as np

In [67]:
Train = pd.read_excel('df_final.xlsx')

In [68]:
from sklearn.model_selection import train_test_split

In [69]:
Train.head()

,Unnamed: 0,20분당시청률,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,...,가을,겨울,봄,여름,20분당시청률_Band,"0_(-0.00185, 0.37]","0_(0.37, 0.741]","0_(0.741, 1.111]","0_(1.111, 1.482]","0_(1.482, 1.852]"
0,0,0.0,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000,...,False,True,False,False,"(-0.00185, 0.37]",1,0,0,0,0
1,1,0.0,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000,...,False,True,False,False,"(-0.00185, 0.37]",1,0,0,0,0
2,2,0.0,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000,...,False,True,False,False,"(-0.00185, 0.37]",1,0,0,0,0
3,3,0.0,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,9337000,...,False,True,False,False,"(-0.00185, 0.37]",1,0,0,0,0
4,4,0.0,2019-01-01 06:40:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,6819000,...,False,True,False,False,"(-0.00185, 0.37]",1,0,0,0,0


In [70]:
Train[['월', '화', '수', '목', '금', '토','일', 
    'month', 'day', 'hour', 'resting_day', 'working_day', 
    '가구', '가전', '건강기능', '농수축', '생활용품', '속옷', '의류', '이미용', '잡화', '주방', '침구',
    '가을', '겨울', '봄', '여름']] = Train[['월', '화', '수', '목', '금', '토','일', 
    'month', 'day', 'hour', 'resting_day', 'working_day', 
    '가구', '가전', '건강기능', '농수축', '생활용품', '속옷', '의류', '이미용', '잡화', '주방', '침구',
    '가을', '겨울', '봄', '여름']] * 1

In [71]:
Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36899 entries, 0 to 36898
Data columns (total 48 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Unnamed: 0          36899 non-null  int64         
 1   20분당시청률             36899 non-null  float64       
 2   방송일시                36899 non-null  datetime64[ns]
 3   노출(분)               36899 non-null  float64       
 4   마더코드                36899 non-null  int64         
 5   상품코드                36899 non-null  int64         
 6   상품명                 36899 non-null  object        
 7   상품군                 36899 non-null  object        
 8   판매단가                36899 non-null  int64         
 9   취급액                 36899 non-null  int64         
 10  주문량                 36899 non-null  float64       
 11  월                   36899 non-null  int32         
 12  화                   36899 non-null  int32         
 13  수                   36899 non-null  int32     

In [84]:
x = Train.drop(['취급액','상품군','주문량','상품명','Unnamed: 0','방송일시','20분당시청률_Band',
                'season'],axis = 1)
y = Train['취급액']

In [85]:
xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size = 0.3, random_state = 924)

In [86]:
xTrain.shape

(25829, 40)

In [87]:
yTrain.shape

(25829,)

In [88]:
xTest.shape

(11070, 40)

In [89]:
yTest.shape

(11070,)

In [90]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
myparam = { 
'n_estimators':[21,51,101,201,],
'min_samples_split':[2,4,6,8,10],
'max_depth':[3,5,7,9,11,13]}

In [91]:
rfModel =  RandomForestRegressor(n_estimators=151,
                              max_depth=13,
                              min_samples_split=10,
                              n_jobs=2,
                              random_state=42)

In [26]:
gridCV = GridSearchCV(rfModel, param_grid = myparam, cv = 5, n_jobs = 3)
gridCV.fit(xTrain,yTrain)
print(gridCV.best_params_)
gridCV.best_score_

{'max_depth': 13, 'min_samples_split': 10, 'n_estimators': 51}


0.5261398968689216

In [92]:
rfModel.fit(xTrain, yTrain)

RandomForestRegressor(max_depth=13, min_samples_split=10, n_estimators=151,
                      n_jobs=2, random_state=42)

In [93]:
pred = rfModel.predict(xTest)

In [94]:
pred.shape
rfModel.score(xTrain,yTrain)
# 0.645

0.6543929801469792

In [ ]:
y_True = yTest.to_numpy()
y_True

In [ ]:
pred

In [ ]:
def MAPE(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

MAPE(y_True, pred)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mean_absolute_percentage_error(y_True,pred)

In [ ]:
predictions = rfModel.predict(xTest)

# Calculate the absolute errors
errors = abs(predictions - y_True)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
mape = 100 * (errors / y_True)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

mean_absolute_percentage_error(yTest,pred)